ToDo:

Plot important dates against our output: https://www.policyuncertainty.com/media/US_Annotated_Series.pdf

1) Bush Election
2) 9/11
3) Gulf War II
4) Stimulus Debate ~2007/ 2008
5) Lehman/ TARP ~2008/ 2009
6) Eurozone Crisis/ US Midterm Elections ~Nov 2010
7) Debt Ceiling Debate ~2011/2012
8) Fiscal Cliff ~2013
9) Govt. Shutdown ~2013/ 2014
10) Brexit ~2016
11) Trump Election ~Nov 2016
12) US out of TPP ~2016/ 2017
13) Tarrifs 2018

Market Dates: https://www.timetoast.com/timelines/important-dates-in-the-history-of-the-u-s-stock-markets

1) Dot Com Bubble = Mar 10, 2000
2) NASDAQ Drop 356 Points = Apr 14, 2000
3) Hurricane Katrina = Aug 25, 2005
4) 2008 Market Selloff = 09/29/2008, 10/08/2008, 10/15/2008, 10/22/2008, 12/01/2008, 12/10/2008
5) 2011 Market Selloff = 08/04/2011, 08/08/2011, 09/17/2011, 12/31/2011
6) Derive other dates from SPX data

https://fraser.stlouisfed.org/timeline/covid-19-pandemic

https://fraser.stlouisfed.org/timeline/financial-crisis

In [1]:
%config Completer.use_jedi = False #for intellisense compatibility w/ Jupyter Notebook
import pandas as pd
import ipdb
import copy
from iGetByWithALittle import find_target_columns, find_new_obs_percent_chg, find_index_in_Excel_File_dict

Read in and Format Vintage GDP Data from ALFRED File

In [2]:
#URL: https://alfred.stlouisfed.org/series/downloaddata?seid=GDPC1
#Note: Must select 'All' in the Vintage Dates section. Data as of 1/9/2021
dfA = pd.read_excel(io='GDPC1_2.xls', sheet_name=1) #read in first sheet of vintages
dfB = pd.read_excel(io='GDPC1_2.xls', sheet_name=2) #read in second sheet of vintages
dfA.set_index('observation_date', inplace=True) #set index to observation date for join
dfB.set_index('observation_date', inplace=True) #set index to observation date for join
df = dfA.join(dfB) #join sheets' data to single data frame
df = df.loc['1999-01-01':] #discard data prior to 1999 as our series begins in 2000
years_cols_list = find_target_columns(df, 6, 10, 1999) #find columns where year >= 1999
df = df.filter(years_cols_list, axis=1) #filter df on columns list
df = df.drop(columns="GDPC1_20031210") #drop 2003 benchmark revision without last observation. this should be static though it wouldn't hurt to write a function to id 'shorter' columns as it iterates
df['Curr GDP Pct Chg'] = find_new_obs_percent_chg(df, 4, 0) #find the percent change given new quarterly observations
GDPC1_cols_list = find_target_columns(df, 0, 5, 'GDPC1') #find columns which contain 'GDPC1'
GDP_actuals = df.drop(columns=GDPC1_cols_list) #remove levels data
GDP_actuals['Prev GDP Pct Chg'] = GDP_actuals['Curr GDP Pct Chg'].shift() #lag previous periods GDP percent change
GDP_actuals = GDP_actuals[['Prev GDP Pct Chg', 'Curr GDP Pct Chg']] #list to define column order
GDP_actuals.index = GDP_actuals.index.to_period("Q") #format index
GDP_actuals.index.rename('Period', inplace = True) #rename index

Read in and Format Vintage GDP Chain-Type Price Index Data from ALFRED File

In [3]:
#URL: https://alfred.stlouisfed.org/series/downloaddata?seid=GDPCTPI
#NOTE: Must select 'All' in the Vintage Dates section. Data as of 1/12/2021
dfA = pd.read_excel(io='GDPCTPI_2.xls', sheet_name=1) #read in first sheet of vintages
dfB = pd.read_excel(io='GDPCTPI_2.xls', sheet_name=2) #read in second sheet of vintages
dfA.set_index('observation_date', inplace=True) #set index to observation date for join
dfB.set_index('observation_date', inplace=True) #set index to observation date for join
df = dfA.join(dfB) #join sheets' data to single data frame
df = df.loc['1999-01-01':] #discard data prior to 1999 as our series begins in 2000
years_cols_list = find_target_columns(df, 8, 12, 1999) #find columns where year >= 1999
df = df.filter(years_cols_list, axis=1) #filter df on columns list
df = df.drop(columns="GDPCTPI_20031210") #drop 2003 benchmark revision without last observation. this should be static though it wouldn't hurt to write a function to id 'shorter' columns as it iterates
df['Curr PGDP Pct Chg'] = find_new_obs_percent_chg(df, 4, 0) #find the percent change given new quarterly observations
GDPCTPI_cols_list = find_target_columns(df, 0, 5, 'GDPCTPI') #find columns which contain 'GDPC1'
PGDP_actuals = df.drop(columns=GDPCTPI_cols_list) #remove levels data
PGDP_actuals['Prev PGDP Pct Chg'] = PGDP_actuals['Curr PGDP Pct Chg'].shift() #lag previous periods GDP percent change
PGDP_actuals = PGDP_actuals[['Prev PGDP Pct Chg', 'Curr PGDP Pct Chg']] #list to define column order
PGDP_actuals.index = PGDP_actuals.index.to_period("Q") #format index
PGDP_actuals.index.rename('Period', inplace = True) #rename index

Read in and Format Vintage PCE Data from ALFRED File

In [4]:
#URL: https://alfred.stlouisfed.org/series/downloaddata?seid=PCEPILFE
#Note: Must select 'All' in the Vintage Dates section. Data as of 1/9/2021
# Might want to dig a little harder here. Could be a better vintage series...
df = pd.read_excel(io='PCEPILFE_2.xls', sheet_name=1) #read in data
df.set_index('observation_date', inplace=True) #set index
df = df.loc['2006-01-01':] #filter extra data
PCE_cols_list = find_target_columns(df, 9, 13, 2006) #identify columns we want to retain
df = df.filter(PCE_cols_list, axis=1) #retain identified columns
df = df.resample('Q').sum(min_count=3)/3 #resample monthly data to quarterly to match Philly convention
df['Curr PCE Pct Chg'] = find_new_obs_percent_chg(df, 1, 1) #find vintage percent changes to match Philly convention 
PCEPI_cols_list = find_target_columns(df, 0, 5, 'PCEPI') #identify columns to remove
PCE_actuals = df.drop(columns=PCEPI_cols_list) #remove identified columns
PCE_actuals['Prev PCE Pct Chg'] = PCE_actuals['Curr PCE Pct Chg'].shift() #create lag of Current values for Previous
PCE_actuals = PCE_actuals[PCE_actuals['Prev PCE Pct Chg'].notna()] #remove unpopulated data after lag
PCE_actuals = PCE_actuals.shift(-1) #lag values to correct index (as Philly Fed index is to the forecast, not GDP period) 
PCE_actuals = PCE_actuals[['Prev PCE Pct Chg', 'Curr PCE Pct Chg']] #set order of columns
PCE_actuals = PCE_actuals.loc['2007-03-31':] #remove unpopulated data
PCE_actuals.index = PCE_actuals.index.to_period("Q") #format index
PCE_actuals.index.rename('Period', inplace = True) #rename index
PCE_actuals.dropna(subset=['Curr PCE Pct Chg'], inplace=True)

Read in and Format US Economic Policy Uncertainty Index Data from ALFRED

In [65]:
#URL: https://alfred.stlouisfed.org/series/downloaddata?seid=USEPUINDXD
#Note: Cannot select all here as too many vintages exists.
#      Vintages for this script are 'beginning - 2017-12-29' and '2018-01-03 - Current'
#      EPU also has a Categorical dataset which has a items that we could combine to filter out monetary policy

# open workbooks
xls1 = pd.ExcelFile('USEPUINDXD_2 - 2014 - 2017-12-29.xls')
xls2 = pd.ExcelFile('USEPUINDXD_2 - 2018 - Current.xls')

# store files' contents in dictionaries
EPU_dict = pd.read_excel(xls1, sheet_name=None)
EPU_dict2 = pd.read_excel(xls2, sheet_name=None)

# combine dictionaries 
EPU_dict.update(EPU_dict2)

# set index of all Vintage dataframes in dict.values to the observation date column
[EPU_dict[i].set_index('observation_date', inplace=True) for i in EPU_dict.keys() 
     if i[0:7]=='Vintage']

# find the index from one of the vintage dataframes
df_index = find_index_in_Excel_File_dict(EPU_dict)

# initialize a new df with the index found above
EPU_df = pd.DataFrame(index=df_index)

# combine vintage dfs into master df
for key, value in EPU_dict.items():
    if key[0:7] == 'Vintage':
        EPU_df = EPU_df.join(EPU_dict1[key])

In [66]:
EPU_df_copy = EPU_df.copy()

In [67]:
EPU_df_copy = EPU_df_copy.loc['2000-01-01':] #filter extra data
# df = df.resample('Q').sum(min_count=3)/3 #resample monthly data to quarterly to match Philly convention
# df['Curr PCE Pct Chg'] = find_new_obs_percent_chg(df, 1, 1) #find vintage percent changes to match Philly convention 
# PCEPI_cols_list = find_target_columns(df, 0, 5, 'PCEPI') #identify columns to remove
# PCE_actuals = df.drop(columns=PCEPI_cols_list) #remove identified columns
# PCE_actuals['Prev PCE Pct Chg'] = PCE_actuals['Curr PCE Pct Chg'].shift() #create lag of Current values for Previous
# PCE_actuals = PCE_actuals[PCE_actuals['Prev PCE Pct Chg'].notna()] #remove unpopulated data after lag
# PCE_actuals = PCE_actuals.shift(-1) #lag values to correct index (as Philly Fed index is to the forecast, not GDP period) 
# PCE_actuals = PCE_actuals[['Prev PCE Pct Chg', 'Curr PCE Pct Chg']] #set order of columns
# PCE_actuals = PCE_actuals.loc['2007-03-31':] #remove unpopulated data
# PCE_actuals.index = PCE_actuals.index.to_period("Q") #format index
# PCE_actuals.index.rename('Period', inplace = True) #rename index
# PCE_actuals.dropna(subset=['Curr PCE Pct Chg'], inplace=True)

,USEPUINDXD_20140327,USEPUINDXD_20140328,USEPUINDXD_20140331,USEPUINDXD_20140401,USEPUINDXD_20140402,USEPUINDXD_20140403,USEPUINDXD_20140404,USEPUINDXD_20140407,USEPUINDXD_20140408,USEPUINDXD_20140409,...,USEPUINDXD_20201231,USEPUINDXD_20210104,USEPUINDXD_20210105,USEPUINDXD_20210106,USEPUINDXD_20210107,USEPUINDXD_20210108,USEPUINDXD_20210111,USEPUINDXD_20210112,USEPUINDXD_20210113,USEPUINDXD_20210114
observation_date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,68.03945,68.03945,68.03945,68.03945,68.03945,68.03945,68.03945,68.03945,68.03945,68.03945,...,68.04,68.04,68.04,68.04,68.04,68.04,68.04,68.04,68.04,68.04
2000-01-02,119.35917,119.35917,119.35917,119.35917,119.35917,119.35917,119.35917,119.35917,119.35917,119.35917,...,119.36,119.36,119.36,119.36,119.36,119.36,119.36,119.36,119.36,119.36
2000-01-03,35.73232,35.73232,35.73232,35.73232,35.73232,35.73232,35.73232,35.73232,35.73232,35.73232,...,35.73,35.73,35.73,35.73,35.73,35.73,35.73,35.73,35.73,35.73
2000-01-04,109.30513,109.30513,109.30513,109.30513,109.30513,109.30513,109.30513,109.30513,109.30513,109.30513,...,109.31,109.31,109.31,109.31,109.31,109.31,109.31,109.31,109.31,109.31
2000-01-05,123.22074,123.22074,123.22074,123.22074,123.22074,123.22074,123.22074,123.22074,123.22074,123.22074,...,123.22,123.22,123.22,123.22,123.22,123.22,123.22,123.22,123.22,123.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,136.08,128.46,142.67,141.50
2021-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,297.15,279.25,290.29,290.14
2021-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,435.47,341.95,325.10


Read in and Format Mean GDP Forecast Data from FRB Philadelphia

In [5]:
#URL: https://www.philadelphiafed.org/surveys-and-data/rgdp
#Note: 'Mean Responses' as of 1/10/2021
df = pd.read_excel(io='Mean_RGDP_Level.xlsx', sheet_name='Mean_Level') #read in data
df = df[df['YEAR'].notna()] #drop last col which shows up as NaT in index (not sure why this shows up...)
df['YEAR'] = df['YEAR'].astype(int).astype(str) #cast year to string for Period index
df['QUARTER'] = df['QUARTER'].astype(int).astype(str) #cast quarter to string for Period index
df['Period'] = df[['YEAR', 'QUARTER']].agg('-Q'.join, axis=1) #combine year and quarter data into Period
df['Period'] = pd.to_datetime(df['Period']) #set Period to datetime
df.set_index('Period', inplace=True) #set Period as df index
df.index = df.index.to_period("Q") #format index
GDP_cols_list = ['RGDP1', 'RGDP2', 'RGDP3'] #identify columns we want to retain and lag
df = df.filter(GDP_cols_list, axis=1) #retain columns we want
df[GDP_cols_list] = df[GDP_cols_list].shift(-1) #lag values to correct index (as Philly Fed index is to the forecast, not GDP period
df = df[df['RGDP1'].notna()] #remove rows where RGDP1 has NaN
df['1Q Ahead Mean GDP Forecast'] = df[['RGDP1', 'RGDP2']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast 1evels 
df['2Q Ahead Mean GDP Forecast'] = df[['RGDP2', 'RGDP3']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast levels 
mean_GDP_forecasts = ['1Q Ahead Mean GDP Forecast', '2Q Ahead Mean GDP Forecast'] #identify columns to retain
mean_GDP_forecasts = df.filter(mean_GDP_forecasts, axis=1) #filter for wanted columns
# mean_GDP_forecasts

C:\Users\micha\miniconda3\envs\FOMC\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Read in and Format Median GDP Forecast Data from FRB Philadelphia

In [6]:
#URL: https://www.philadelphiafed.org/surveys-and-data/rgdp
#Note: 'Median Responses' as of 1/10/2021
df = pd.read_excel(io='Median_RGDP_Level.xlsx', sheet_name='Median_Level') #read in data
df = df[df['YEAR'].notna()] #drop last col which shows up as NaT in index (not sure why this shows up...)
df['YEAR'] = df['YEAR'].astype(int).astype(str) #cast year to string for Period index
df['QUARTER'] = df['QUARTER'].astype(int).astype(str) #cast quarter to string for Period index
df['Period'] = df[['YEAR', 'QUARTER']].agg('-Q'.join, axis=1) #combine year and quarter data into Period
df['Period'] = pd.to_datetime(df['Period']) #set Period to datetime
df.set_index('Period', inplace=True) #set Period as df index
df.index = df.index.to_period("Q") #format index
GDP_cols_list = ['RGDP1', 'RGDP2', 'RGDP3'] #identify columns we want to retain and lag
df = df.filter(GDP_cols_list, axis=1) #retain columns we want
df[GDP_cols_list] = df[GDP_cols_list].shift(-1) #lag values to correct index (as Philly Fed index is to the forecast, not GDP period) 
df = df[df['RGDP1'].notna()] #remove rows where RGDP1 has NaN
df['1Q Ahead Median GDP Forecast'] = df[['RGDP1', 'RGDP2']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast 1
df['2Q Ahead Median GDP Forecast'] = df[['RGDP2', 'RGDP3']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast 1
median_GDP_forecasts = ['1Q Ahead Median GDP Forecast', '2Q Ahead Median GDP Forecast'] #identify columns to retain
median_GDP_forecasts = df.filter(median_GDP_forecasts, axis=1) #filter for wanted columns
# median_GDP_forecasts

Read in and Format Mean GDP Price Index Forecast Data from FRB Philadelphia

In [7]:
#URL: https://www.philadelphiafed.org/surveys-and-data/pgdp
#Note: 'Mean Responses' as of 1/12/2021
df = pd.read_excel(io='Mean_PGDP_Level.xlsx', sheet_name='Mean_Level') #read in data
df = df[df['YEAR'].notna()] #drop last col which shows up as NaT in index (not sure why this shows up...)
df['YEAR'] = df['YEAR'].astype(int).astype(str) #cast year to string for Period index
df['QUARTER'] = df['QUARTER'].astype(int).astype(str) #cast quarter to string for Period index
df['Period'] = df[['YEAR', 'QUARTER']].agg('-Q'.join, axis=1) #combine year and quarter data into Period
df['Period'] = pd.to_datetime(df['Period']) #set Period to datetime
df.set_index('Period', inplace=True) #set Period as df index
df.index = df.index.to_period("Q") #format index
PGDP_cols_list = ['PGDP1', 'PGDP2', 'PGDP3'] #identify columns we want to retain and lag
df = df.filter(PGDP_cols_list, axis=1) #retain columns we want
df[PGDP_cols_list] = df[PGDP_cols_list].shift(-1) #lag values to correct index (as Philly Fed index is to the forecast, not GDP period
df = df[df['PGDP1'].notna()] #remove rows where RGDP1 has NaN
df['1Q Ahead Mean GDP Price Forecast'] = df[['PGDP1', 'PGDP2']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast 1evels 
df['2Q Ahead Mean GDP Price Forecast'] = df[['PGDP2', 'PGDP3']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast levels 
mean_PGDP_forecasts = ['1Q Ahead Mean GDP Price Forecast', '2Q Ahead Mean GDP Price Forecast'] #identify columns to retain
mean_PGDP_forecasts = df.filter(mean_PGDP_forecasts, axis=1) #filter for wanted columns
# mean_PGDP_forecasts

Read in and Format Median GDP Price Index Forecast Data from FRB Philadelphia

In [8]:
#URL: https://www.philadelphiafed.org/surveys-and-data/pgdp
#Note: 'Median Responses' as of 1/12/2021
df = pd.read_excel(io='Median_PGDP_Level.xlsx', sheet_name='Median_Level') #read in data
#print(df)
df = df[df['YEAR'].notna()] #drop last col which shows up as NaT in index (not sure why this shows up...)
df['YEAR'] = df['YEAR'].astype(int).astype(str) #cast year to string for Period index
df['QUARTER'] = df['QUARTER'].astype(int).astype(str) #cast quarter to string for Period index
df['Period'] = df[['YEAR', 'QUARTER']].agg('-Q'.join, axis=1) #combine year and quarter data into Period
df['Period'] = pd.to_datetime(df['Period']) #set Period to datetime
df.set_index('Period', inplace=True) #set Period as df index
df.index = df.index.to_period("Q") #format index
PGDP_cols_list = ['PGDP1', 'PGDP2', 'PGDP3'] #identify columns we want to retain and lag
df = df.filter(PGDP_cols_list, axis=1) #retain columns we want
df[PGDP_cols_list] = df[PGDP_cols_list].shift(-1) #lag values to correct index (as Philly Fed index is to the forecast, not GDP period
df = df[df['PGDP1'].notna()] #remove rows where RGDP1 has NaN
df['1Q Ahead Median GDP Price Forecast'] = df[['PGDP1', 'PGDP2']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast 1evels 
df['2Q Ahead Median GDP Price Forecast'] = df[['PGDP2', 'PGDP3']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast levels 
median_PGDP_forecasts = ['1Q Ahead Median GDP Price Forecast', '2Q Ahead Median GDP Price Forecast'] #identify columns to retain
median_PGDP_forecasts = df.filter(median_PGDP_forecasts, axis=1) #filter for wanted columns
# median_PGDP_forecasts

Read in and Format Mean PCE Forecast Data from FRB Philadelphia

In [9]:
#URL: https://www.philadelphiafed.org/surveys-and-data/corepce
#Note: 'Mean Responses' as of 1/10/2021
df = pd.read_excel(io='Mean_COREPCE_Level.xlsx', sheet_name='Mean_Level') #read in data
df = df[df['YEAR'].notna()] #drop last col which shows up as NaT in index (not sure why this shows up...)
df['YEAR'] = df['YEAR'].astype(int).astype(str) #cast year to string for Period index
df['QUARTER'] = df['QUARTER'].astype(int).astype(str) #cast quarter to string for Period index
df['Period'] = df[['YEAR', 'QUARTER']].agg('-Q'.join, axis=1) #combine year and quarter data into Period
df['Period'] = pd.to_datetime(df['Period']) #set Period to datetime
df.set_index('Period', inplace=True) #set Period as df index
df.index = df.index.to_period("Q") #format index
PCE_cols_list = ['COREPCE1', 'COREPCE2', 'COREPCE3'] #identify columns we want to retain and lag
df[PCE_cols_list] = df[PCE_cols_list].shift(-1) #lag values to correct index (as Philly Fed index is to the forecast, not GDP period
df = df[df['COREPCE1'].notna()] #remove rows where COREPCE1 has NaN
df = df.filter(PCE_cols_list, axis=1) #retain columns we want
df.rename(columns={'COREPCE1':'COREPCE - Mean','COREPCE2':'1Q Ahead Mean PCE Forecast','COREPCE3':'2Q Ahead Mean PCE Forecast'}, inplace=True) #rename column
mean_PCE_forecasts = ['COREPCE - Mean', '1Q Ahead Mean PCE Forecast', '2Q Ahead Mean PCE Forecast'] #identify columns to retain *keeping mean for actuals comparison* 
mean_PCE_forecasts = df.filter(mean_PCE_forecasts, axis=1) #filter for wanted columns
# mean_PCE_forecasts

Read in and Format Median PCE Forecast Data from FRB Philadelphia

In [10]:
#URL: https://www.philadelphiafed.org/surveys-and-data/corepce
#Note: 'Median Responses' as of 1/10/2021
df = pd.read_excel(io='Median_COREPCE_Level.xlsx', sheet_name='Median_Level') #read data
df = df[df['YEAR'].notna()] #drop last col which shows up as NaT in index (not sure why this shows up...)
df['YEAR'] = df['YEAR'].fillna(0.0).astype(int).astype(str) #cast year to string for Period index
df['QUARTER'] = df['QUARTER'].fillna(0.0).astype(int).astype(str) #cast quarter to string for Period index
df['Period'] = df[['YEAR', 'QUARTER']].agg('-Q'.join, axis=1) #combine year and quarter data into Period
df['Period'] = pd.to_datetime(df['Period']) #set Period to datetime
df.set_index('Period', inplace=True) #set Period as df index
df.index = df.index.to_period("Q") #format index
PCE_cols_list = ['COREPCE1', 'COREPCE2', 'COREPCE3'] #identify columns we want to retain and lag
df[PCE_cols_list] = df[PCE_cols_list].shift(-1) #lag values to correct index (as Philly Fed index is to the forecast, not GDP period
df = df[df['COREPCE1'].notna()] #remove rows where COREPCE1 has NaN
df = df.filter(PCE_cols_list, axis=1) #retain columns we want
df.rename(columns={'COREPCE1':'COREPCE - Median','COREPCE2':'1Q Ahead Median PCE Forecast','COREPCE3':'2Q Ahead Median PCE Forecast'}, inplace=True) #rename column
median_PCE_forecasts = ['COREPCE - Median', '1Q Ahead Median PCE Forecast', '2Q Ahead Median PCE Forecast']
median_PCE_forecasts = df.filter(median_PCE_forecasts, axis=1)
# median_PCE_forecasts

Read in and Format S&P500 Data from Yahoo

Read in and Format US Political Uncertainty Index Data from ALFRED

Combine GDP Data from ALFRED & Philly Fed

In [11]:
GDP_data = GDP_actuals.join(mean_GDP_forecasts) #left join the GDP_acuals and mean_GDP_forecasts
GDP_data = GDP_data.join(median_GDP_forecasts) #left join the GDP_acuals and median_GDP_forecasts
GDP_data = GDP_data.loc['2000Q1':] #remove rows outside of our period
GDP_data.index = GDP_data.index.strftime('%m/%d/%Y') #set index format to m/d/yyyy

Combine PCE Data from ALFRED & Philly Fed

In [12]:
PCE_forecasts = mean_PCE_forecasts.join(median_PCE_forecasts)
PCE_data = PCE_actuals.join(PCE_forecasts)
# # Test for data continuity
# print('Corr Median & FRED Actual', PCE_data['Curr PCE Pct Chg'].corr(PCE_data['COREPCE - Median'])) #check for data continuity across the FRED actual and Philly Fed forcast data
# print('Corr Median & FRED Actual', PCE_data['Curr PCE Pct Chg'].corr(PCE_data['COREPCE - Mean'])) #check for data continuity across the FRED actual and Philly Fed forcast data
PCE_data_to_drop = ['COREPCE - Median','COREPCE - Mean']
PCE_data = PCE_data.drop(PCE_data_to_drop, axis=1)
PCE_data.index = PCE_data.index.strftime('%m/%d/%Y') #set index format to m/d/yyyy

Combine PGDP Data from ALFRED & Philly Fed

In [13]:
PGDP_forecasts = mean_PGDP_forecasts.join(median_PGDP_forecasts)
PGDP_data = PGDP_actuals.join(PGDP_forecasts)
PGDP_data = PGDP_data.loc['2000Q1':] #remove rows outside of our period
PGDP_data.index = PGDP_data.index.strftime('%m/%d/%Y') #set index format to m/d/yyyy

Combine Datasets for Shock Regression

In [14]:
'''
    Miles, this one is for you. Call the ball. :)
'''

macro_data = GDP_data.join(PGDP_data)
# macro_data
# macro_data.to_csv('macro_data.csv')